# Comparing different parameters for QAOA applied to MaxCut

Different parameters have been proposed for QAOA applied to MaxCut. Two notable examples are the "fixed angle conjecture" parameters of [1] and the parameters obtained for the SK model in the infinite size limit [2]. Both are available in QOKit. Below we show an example of comparing QAOA performance with the two parameter setting strategies.

[1] The fixed angle conjecture for QAOA on regular MaxCut graphs. Jonathan Wurtz, Danylo Lykov. https://arxiv.org/abs/2107.00677

[2] The Quantum Approximate Optimization Algorithm at High Depth for MaxCut on Large-Girth Regular Graphs and the Sherrington-Kirkpatrick Model. Joao Basso, Edward Farhi, Kunal Marwaha, Benjamin Villalonga, Leo Zhou. https://arxiv.org/abs/2110.14206

In [ ]:
import numpy as np
import networkx as nx
from functools import partial

from qokit.qaoa_objective_maxcut import get_qaoa_maxcut_objective
from qokit.maxcut import maxcut_obj, get_adjacency_matrix
from qokit.utils import precompute_energies
from qokit.parameter_utils import get_fixed_gamma_beta, get_sk_gamma_beta

In [ ]:
import warnings
warnings.simplefilter("ignore", UserWarning)

In [ ]:
# Generate a random regular graph
N = 10
d = 5
seed = 1
G = nx.random_regular_graph(d,N,seed=seed)

# precompute energies to speed up computation
obj = partial(maxcut_obj, w=get_adjacency_matrix(G))
precomputed_energies = precompute_energies(obj, N)
optimal_cut = np.max(precomputed_energies)

for p in range(1, 17):
    print(f"QAOA Approximation Ratio at p = {p}")
    f = get_qaoa_maxcut_objective(N, p, precomputed_cuts=precomputed_energies, parameterization='gamma beta')
    try:
        gamma, beta = get_fixed_gamma_beta(d, p)
        print(f'Fixed: {-f(gamma, beta) / optimal_cut}')
    except ValueError:
        # skip if parameters not available
        pass
    try:
        gamma, beta = get_sk_gamma_beta(p)
        print(f'SK: {-f(gamma / np.sqrt(d), beta) / optimal_cut}')
    except ValueError:
        # skip if parameters not available
        pass